In [1]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from typing import Callable, List, Tuple

# To ignore LightGBM parameter warnings
import warnings
warnings.filterwarnings("ignore")

In [2]:
data = pd.read_csv('data/train.csv', parse_dates=['Date'])
stores = pd.read_csv('data/store.csv')
store_features = stores[['Store', 'StoreType', 'Assortment', 'CompetitionDistance', 'Promo2']]

data = \
(
    data
    .merge(store_features.set_index('Store'), on='Store', how='left')  # Add store features
    .loc[data.Open == 1]  # We don't need closed stores data for our purposes
    .drop(columns=['Open'])
    .sort_values('Date', ascending=True)  # We'll need our values sorted for later
)
data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2
1017190,1097,2,2013-01-01,5961,1405,0,a,1,b,b,720.0,0
1016179,85,2,2013-01-01,4220,619,0,a,1,b,a,1870.0,0
1016353,259,2,2013-01-01,6851,1444,0,a,1,b,b,210.0,0
1016356,262,2,2013-01-01,17267,2875,0,a,1,b,a,1180.0,0
1016368,274,2,2013-01-01,3102,729,0,a,1,b,b,3640.0,1


In [3]:
# Borders to split our data in train/val/split
val_border = '2015-04-01'
test_border = '2015-05-01'

# Statistics we'll use as additional features
stats = ['mean', 'std', 'min', 'max']

In [4]:
# We expect a very pronounced seasonality in drug store sales data
# Simply adding a month feature may be enough for this demo
data['month'] = data.Date.dt.month

# To indicate LightGBM that these are the categorical features in our data
data[['DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 'month']] =\
data[['DayOfWeek', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'Promo2', 'month']].astype('category')
data.head(5)

,Store,DayOfWeek,Date,Sales,Customers,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,Promo2,month
1017190,1097,2,2013-01-01,5961,1405,0,a,1,b,b,720.0,0,1
1016179,85,2,2013-01-01,4220,619,0,a,1,b,a,1870.0,0,1
1016353,259,2,2013-01-01,6851,1444,0,a,1,b,b,210.0,0,1
1016356,262,2,2013-01-01,17267,2875,0,a,1,b,a,1180.0,0,1
1016368,274,2,2013-01-01,3102,729,0,a,1,b,b,3640.0,1,1


In [5]:
def merge_last_statistics(df_from: pd.DataFrame, df_to: pd.DataFrame, on_column: str, cols=None) -> pd.DataFrame:
    ''' Returns @df_to with merged unique @df_from columns '''
    # Select only unique @df_from columns if not specified explicitly
    cols = cols if cols is not None else df_from.columns.difference(df_to.columns)
    result = \
    (
        df_to
        .merge(
            df_from
                .groupby(on_column, as_index=True)  # as_index=True for an upcoming merge
                [cols]
                .last(),  # Leave only the last entry. It's assumed that df_from is in correct order
            on=on_column,
            how='left'
        )
    )
    return result

In [6]:
def calculate_statistics_naive(data: pd.DataFrame, stats: List[str]) -> pd.DataFrame:
    ''' Implements first (naive) method of adding statistics to the data '''
    data = data.copy()
    for s in stats:
        data[s + '_sales'] = data.groupby('Store').Sales.transform(s)
        data[s + '_customers'] = data.groupby('Store').Customers.transform(s)
    return data

In [7]:
def calculate_statistics_no_leak(data: pd.DataFrame, stats: List[str]) -> pd.DataFrame:
    ''' Implements second (no direct test -> train leak) method '''
    data = data.copy()
        
    for s in stats:
        data[s + '_sales'] = data.groupby('Store').Sales.transform(
            lambda x: x.shift().expanding().aggregate(s)
        )
        data[s + '_customers'] = data.groupby('Store').Customers.transform(
            lambda x: x.shift().expanding().aggregate(s)
        )
            
    return data

In [8]:
# Zero sales data was ignored in the competition, so it would just make noise for our simple model
# Moreover we use RMSPE metric
data = data.loc[data.Sales != 0]

# train/val/test split
train = data.loc[data.Date <= val_border]
validation = data.loc[(data.Date > val_border) & (data.Date < test_border)]
test = data.loc[data.Date >= test_border]

In [9]:
train = calculate_statistics_no_leak(train, stats)
latest_stats = calculate_statistics_no_leak(data.drop(test.index), stats)

# train = calculate_statistics_naive(train, stats)
# latest_stats = calculate_statistics_naive(data.drop(test.index), stats)

validation = merge_last_statistics(train, validation, 'Store')
test = merge_last_statistics(latest_stats, test, 'Store')

train = train.drop(columns=['Customers'])
validation = validation.drop(columns=['Customers'])

# That is to preserve train order of columns after merging them into val/test
validation = validation[train.columns]
test = test[train.columns]

In [10]:
def make_feval(f: Callable, name: str = None, higher_better: bool = False) -> Callable:
    """
    Function factory to transform @f to @feval required by LightGBM
    Args:
        f: function of 2 arguments (predictions, true_values) -> score
        name: name of function (f.name will be used if None)
        higher_better: True if higher score is better, otherwise False
    Returns:
        feval: function of 2 arguments (predictions, Dataset with true labels) -> (name, score, higher_better)
    """

    def feval(X: np.ndarray, Y: lgb.Dataset) -> Tuple[str, Callable, bool]:
        return name if name is not None else f.__name__,\
               f(X, Y.get_label()),\
               higher_better

    return feval

def RMSPE(X: np.ndarray, Y: np.ndarray) -> np.float64:
    return np.sqrt(np.mean(np.square(((Y - X) / Y))))

def wMAPE(X: np.ndarray, Y: np.ndarray) -> np.float64:
    return np.sum(np.absolute(Y - X)) / np.sum(np.absolute(Y))

In [11]:
# LightGBM abstraction of a Dataset
validation_set = lgb.Dataset(validation.drop(columns=['Sales', 'Date']), validation['Sales'])
train_set = lgb.Dataset(train.drop(columns=['Sales', 'Date']), train['Sales'])

# We don't need abstractions for test data
test_X = test.drop(columns=['Sales', 'Date'])
test_Y = test['Sales']

params = {
    'objective': 'poisson',  # Usually works better with sales data
    'metric': 'mae',  # For fancier runtime reports
    'num_iterations': 5000,  # Early stopping is supposed to stop training procedure earlier
    'early_stopping_round': 30
}

model = lgb.train(
    params,
    train_set=train_set,
    valid_sets=[validation_set],
    feval=make_feval(RMSPE),
    verbose_eval=100
)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.011381 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2591
[LightGBM] [Info] Number of data points in the train set: 734032, number of used features: 18
[LightGBM] [Info] Start training from score 8.841784
Training until validation scores don't improve for 30 rounds
[100]	valid_0's l1: 937.761	valid_0's RMSPE: 0.206806
[200]	valid_0's l1: 895.512	valid_0's RMSPE: 0.189244
[300]	valid_0's l1: 874.266	valid_0's RMSPE: 0.173872
[400]	valid_0's l1: 860.442	valid_0's RMSPE: 0.169213
[500]	valid_0's l1: 850.953	valid_0's RMSPE: 0.166557
[600]	valid_0's l1: 842.835	valid_0's RMSPE: 0.164156
[700]	valid_0's l1: 835.478	valid_0's RMSPE: 0.162334
[800]	valid_0's l1: 830.092	valid_0's RMSPE: 0.161204
[900]	valid_0's l1: 825.655	valid_0's RMSPE: 0.159525
[1000]	valid_0's l1: 820.774	valid_0's RMSP

## This pipeline is expected to produce the following output
### For naive method
RMSPE(test_Prediction, test_Y)=0.14764652825456756  
wMAPE(test_Prediction, test_Y)=0.1101648237569947
### For method with no leaks
RMSPE(test_Prediction, test_Y)=0.13690783985271288  
wMAPE(test_Prediction, test_Y)=0.09835009306031911

In [12]:
test_Prediction = model.predict(test_X)
print(f'{RMSPE(test_Prediction, test_Y)=}')
print(f'{wMAPE(test_Prediction, test_Y)=}')

RMSPE(test_Prediction, test_Y)=0.13690783985271288
wMAPE(test_Prediction, test_Y)=0.09835009306031911


## Tuning the model
You may want to tune both models and compare scores then. 
For that, I recommend using [Optuna](https://optuna.org/) integration with LightGBM:  
- Install additional package by uncommenting and running the cell below
- Change `import lightgbm as lgb` to `import optuna.integration.lightgbm as lgb`
- Rerun the notebook

## I've already done that and got the following results:

### For naive method
RMSPE(test_Prediction, test_Y)=0.14168013535199925  
wMAPE(test_Prediction, test_Y)=0.10707387121191184
### For method with no leaks
RMSPE(test_Prediction, test_Y)=0.12982188652207252  
wMAPE(test_Prediction, test_Y)=0.09360049370506834

In [13]:
# !pip install optuna